# SmartMeter Log Analysis

**Run analysis:** *[Menu] Kernel -> Restart Kernel and Run All Cells...*

This notebook analyzes the SmartMeter camera system logfile for events and compute statistics of repeated activity phases. The logfile contains events of the following activities named after their scripts:

* *fileservice.sh*: picture upload and file management
* *logrotate.sh*: logfile upload and file management
* *smeter.sh*: all activities

The *smeter.sh* activity contains all other.

**Download and prepare logfiles**

The path to `filterlog.sh` script is different on the local dev system and on mybinder.org. We look in different root paths for `filterlog.sh` and once found it gets executed.

In [ ]:
root_paths <- c('/SmartMeter', '..')
filterlog.sh <- 'src/filterlog.sh'
filterlog.sh <- paste0(root_paths, '/', filterlog.sh)
filterlog.sh <- file.access(filterlog.sh, mode=4)
filterlog.sh <- head(filterlog.sh[filterlog.sh == 0], n=1)
filterlog.sh <- names(filterlog.sh)
# run filterlog.sh
system(paste('sh', filterlog.sh, '> smartmeter.log'), intern=TRUE)

In [ ]:
# input
logfile_name <- 'smartmeter.log'

**Import relevant libraries**

In [ ]:
library(tidyverse)
library(lubridate)
library(ggplot2)

### Load logfile

Load and parse csv with `;` as delimiter.

In [ ]:
logfile <- read_delim(file=logfile_name, col_names=FALSE, delim=';')
colnames(logfile) <- c('date', 'time', 'script', 'status', 'message')
head(logfile)

### Prepare data

Filter and convert data types. Convert date and time to timestamp (using lubridate package)

In [ ]:
# convert timestamp
logfile <- logfile %>% 
    mutate(time_sec = str_extract(time, '\\d\\d:\\d\\d:\\d\\d') ) %>%
    mutate(timestamp = ymd_hms(paste(date, time_sec))) %>% 
    select(timestamp, script, status, message)

### Simple stats and plots

Some simple stats figures supported with plots to obtain an overview on all events within a logfile. The statistics are

* event count per status
* file completeness
* timeseries of events
* status per hour

#### Event Count per status

In [ ]:
# event count per status in logfile
# status: INFO, WARN, ERROR
logfile %>% 
    group_by(status) %>%
    summarise(evt_count=n())

#### File completness

We expect 3 file uploads per hour. We only output date and time when there are more or less than 3 files per hour. In the regular case, we expect an empty list.

In [ ]:
# add datehour for each entry
logdatehour <- logfile %>% 
    mutate(datehour=ymd_h(paste(date(timestamp), hour(timestamp))))

In [ ]:
# File completness
# show list only, if there more or less than 3 files per hour
logdatehour %>%
    filter(str_detect(message, 'Upload to Dropbox')) %>%
    filter(str_detect(message, '.png')) %>%
    group_by(datehour) %>%
    summarise(filecount=n()) %>%
    filter(filecount != 3) 
# expect empty list as result

#### Timeseries of events

In [ ]:
# all events per hour
ggplot( logdatehour %>%
    group_by(datehour) %>%
    summarise(evt_hour=n()),
    aes(x=datehour, y=evt_hour)) +
    geom_line(color='blue') +
    geom_point() +
    xlab('Time') +
    ylab('Events per hour') +
    ggtitle('Time series of events in logfile')

#### Timeseries of events for each status

Possible status names are
* INFO
* WARN
* ERROR

In [ ]:
# plot bar chart of each status event counts per hour
ggplot(logdatehour, aes(x=datehour)) +
    geom_bar(alpha=0.2, ) +
    geom_line(stat='count', linetype='dashed', color='blue') +
    geom_point(stat='count') +
    facet_wrap( ~status, scales="free", ncol=1) +
    ggtitle('Barplots of status events per hour') +
    theme_light()

### Activity Duration 

The logfile contains events of the following activities named after their scripts:

* *fileservice.sh*: picture upload and file management
* *logrotate.sh*: logfile upload and file management
* *smeter.sh*: all activities

In this section we filter messages (by msg_str) in log entries (given by script_str) and compute the duration. See function `calc_act_length`.

In [ ]:
# computes the length for selected messages of a script name 
calc_act_length <- function(data, script_str, msg_str) {
    evt_logfile <- data %>%
        filter(str_detect(script, script_str)) %>% # script == script_str?
        filter(str_detect(message, msg_str)) %>% # message == msg_str?
        mutate(act_type = script_str) %>% # set evt_type := script_str 
        arrange(timestamp) # sort asc

    # calc event length
    prev_timestamp <- evt_logfile %>% first %>% lag
    evt_logfile <- tibble(prev_timestamp, evt_logfile) %>% # re-compile logfile for event
        mutate(time_interval = interval(prev_timestamp, timestamp)) %>%
        mutate(interval_sec=int_length(time_interval)) # calc event len in seconds
    
    return(evt_logfile)
}

Now, filter the logfile and extract the begin/end events for activties and compute their respective durations in seconds.

In [ ]:
# smeter.sh 
smeter <- logfile %>% 
    calc_act_length('smeter.sh', 'SmartMeter camera system') %>%
    filter(str_detect(message, 'SmartMeter camera system shuts down'))
# fileservice.sh
fileservice <- logfile %>%
    calc_act_length('fileservice.sh', 'Upload to Dropbox|Delete file') %>%
    filter(str_detect(message, 'Delete file:'))
# logrotate.sh
# this activity happens once per hour (-> sum all events)
logrotate <- logfile %>%
    calc_act_length('logrotate.sh', 'Upload to Dropbox') %>%
    filter(interval_sec < 1000) %>% # hack: filters out the inteval between two cycles
    group_by(act_type, date(timestamp), hour(timestamp)) %>% 
    summarise(interval_sec=sum(interval_sec)) # sum logrotate activity per hour (=cycle)

### Activity duration stats and plots

We compute summary stats for each activity duration and plot them as a boxplot.

In [ ]:
# combine activities
act_duration <- bind_rows(smeter, fileservice, logrotate)

In [ ]:
# descriptive summary stats
act_duration %>% 
    spread(act_type, interval_sec) %>% # convert from long to wide format
    select(smeter.sh, fileservice.sh, logrotate.sh) %>%
    summary

In [ ]:
# cleanup ...
act_duration <- act_duration %>%
    select(act_type, interval_sec) %>% 
    drop_na
# ... and plot
p <- ggplot(act_duration, aes(x=act_type, y=interval_sec, fill=act_type)) +
    geom_boxplot(outlier.colour="red", outlier.shape=8, outlier.size=4) +
    facet_wrap( ~act_type, scales="free", ncol=1) +
    coord_flip() +
    xlab("Activity") +
    ylab("Activity duration [seconds]") +
    guides(fill=guide_legend(title="Activity")) +
    theme_light()
p

In [ ]:
# store plot as png file
png(file='activity_durations.png', width=600)
p
dev.off()